In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score, top_k_accuracy_score
from sklearn.linear_model import LogisticRegression
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential

import warnings

import joblib

# Filter warnings.
warnings.filterwarnings('ignore')

#Load DataSet


In [ ]:
!wget 'https://raw.github.com/Ashwanth-14/Ashwanth/main/Final_Augmented_dataset_Diseases_and_Symptoms.csv.zip'

--2025-07-25 09:20:18--  https://raw.github.com/Ashwanth-14/Ashwanth/main/Final_Augmented_dataset_Diseases_and_Symptoms.csv.zip
Resolving raw.github.com (raw.github.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.github.com (raw.github.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://raw.githubusercontent.com/Ashwanth-14/Ashwanth/main/Final_Augmented_dataset_Diseases_and_Symptoms.csv.zip [following]
--2025-07-25 09:20:18--  https://raw.githubusercontent.com/Ashwanth-14/Ashwanth/main/Final_Augmented_dataset_Diseases_and_Symptoms.csv.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2942107 (2.8M) [application/zip]
Saving to: ‘Final_Augmented_dataset_Disea

In [ ]:
df = pd.read_csv('Final_Augmented_dataset_Diseases_and_Symptoms.csv.zip')

# Analaysis

In [ ]:
# Dataframe shape
df.shape

(246945, 378)

In [ ]:
#Check for duplicates
df.duplicated().value_counts()

,count
False,189647
True,57298


The 600th most common disease is: breast cyst


In [ ]:
# Remove duplicated rows and columns
df = df.drop_duplicates().reset_index(drop=True)

In [ ]:
# Replace spaces with underscores in column names
df.columns = df.columns.str.replace(' ', '_')

In [ ]:
top_diseases = df['diseases'].value_counts().nlargest(500).index
df = df[df['diseases'].isin(top_diseases)]

# Re-check the number of unique diseases
df['diseases'].nunique()

500

In [ ]:
# prompt: drop coloumns where all coloumns have value 0

# Find columns where all values are 0
zero_columns = df.columns[(df == 0).all()]

# Drop these columns
df = df.drop(columns=zero_columns)
df.shape

print(df.head())

         diseases  anxiety_and_nervousness  depression  shortness_of_breath  \
0  panic disorder                        1           0                    1   
1  panic disorder                        0           0                    1   
2  panic disorder                        1           1                    1   
3  panic disorder                        1           0                    0   
4  panic disorder                        1           1                    0   

   depressive_or_psychotic_symptoms  sharp_chest_pain  dizziness  insomnia  \
0                                 1                 0          0         0   
1                                 1                 0          1         1   
2                                 1                 0          1         1   
3                                 1                 0          1         1   
4                                 0                 0          0         1   

   abnormal_involuntary_movements  chest_tightness  ... 

In [ ]:
le = LabelEncoder()
df['label'] = le.fit_transform(df['diseases'])

In [ ]:
# Check result
print(df[['diseases', 'label']].head(10))

         diseases  label
0  panic disorder    340
1  panic disorder    340
2  panic disorder    340
3  panic disorder    340
4  panic disorder    340
5  panic disorder    340
6  panic disorder    340
7  panic disorder    340
8  panic disorder    340
9  panic disorder    340


In [ ]:
df = df.drop(["diseases"], axis=1).copy()
X = df.drop(["label"], axis=1)
y = df["label"]

In [ ]:
X.shape, y.shape

((187592, 316), (187592,))

In [ ]:
num_classes = len(le.classes_)


# MODELS


Neural NetWork


In [ ]:
y_categorical = to_categorical(y)
num_classes = len(le.classes_)

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y_categorical, test_size=0.2, random_state=42)

# Build the model
model = Sequential([
    Dense(512, activation='relu', input_shape=(316,)),   # wide first layer
    Dropout(0.4),                                         # prevent overfitting
    Dense(512, activation='relu'),
    Dropout(0.3),
    Dense(128, activation='relu'),
    Dense(num_classes, activation='softmax')              # output: one per disease
])

# Compile
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train
history = model.fit(X_train, y_train, epochs=30, batch_size=32, validation_split=0.2)

Epoch 1/30
3752/3752 ━━━━━━━━━━━━━━━━━━━━ 38s 10ms/step - accuracy: 0.5366 - loss: 2.1558 - val_accuracy: 0.8086 - val_loss: 0.5378
Epoch 2/30
3752/3752 ━━━━━━━━━━━━━━━━━━━━ 40s 9ms/step - accuracy: 0.7892 - loss: 0.6388 - val_accuracy: 0.8171 - val_loss: 0.4970
Epoch 3/30
3752/3752 ━━━━━━━━━━━━━━━━━━━━ 36s 10ms/step - accuracy: 0.8057 - loss: 0.5728 - val_accuracy: 0.8206 - val_loss: 0.4788
Epoch 4/30
3752/3752 ━━━━━━━━━━━━━━━━━━━━ 36s 10ms/step - accuracy: 0.8126 - loss: 0.5387 - val_accuracy: 0.8221 - val_loss: 0.4627
Epoch 5/30
3752/3752 ━━━━━━━━━━━━━━━━━━━━ 38s 9ms/step - accuracy: 0.8205 - loss: 0.5108 - val_accuracy: 0.8240 - val_loss: 0.4575
Epoch 6/30
3752/3752 ━━━━━━━━━━━━━━━━━━━━ 42s 9ms/step - accuracy: 0.8198 - loss: 0.4998 - val_accuracy: 0.8254 - val_loss: 0.4449
Epoch 7/30
3752/3752 ━━━━━━━━━━━━━━━━━━━━ 36s 10ms/step - accuracy: 0.8235 - loss: 0.4810 - val_accuracy: 0.8259 - val_loss: 0.4407
Epoch 8/30
3752/3752 ━━━━━━━━━━━━━━━━━━━━ 39s 9ms/step - accuracy: 0.8248 - los

In [ ]:
def compute_top3_accuracy(model, X_test, y_test, label_encoder):
    # Predict probabilities
    y_probs = model.predict(X_test, verbose=0)

    # Get top 3 predictions for each sample
    top3_preds = np.argsort(y_probs, axis=1)[:, -3:]

    # Convert true labels if needed
    if not np.issubdtype(y_test.dtype, np.integer):
        y_test = label_encoder.transform(y_test)

    # Check if true label is in top 3
    correct_top3 = np.any(top3_preds == y_test[:, None], axis=1)

    # Compute accuracy
    top3_accuracy = np.mean(correct_top3)

    return round(top3_accuracy * 100, 2)

In [ ]:
# If y_test is one-hot encoded, convert it to label indices
if len(y_test.shape) > 1 and y_test.shape[1] > 1:
    y_test_labels = np.argmax(y_test, axis=1)
else:
    y_test_labels = y_test  # already in correct format

top3_acc = compute_top3_accuracy(model, X_test,y_test_labels, le)
print(f"✅ Top-3 Accuracy: {top3_acc}%")

✅ Top-3 Accuracy: 94.69%


In [ ]:
model.save("my_model.h5")
from google.colab import files
files.download("my_model.h5")




<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
import pickle
with open("label_encoder.pkl", "wb") as f:
    pickle.dump(le, f)

files.download("label_encoder.pkl")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
import pickle
from google.colab import files  # Use only in Google Colab

# Create symptom_index from DataFrame columns
symptom_index = {symptom: idx for idx, symptom in enumerate(X.columns)}

# Save to a file
with open("symptom_index.pkl", "wb") as f:
    pickle.dump(symptom_index, f)

# Download the file (only for Colab)
files.download("symptom_index.pkl")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
print(le.classes_[:10])

['abdominal aortic aneurysm' 'abdominal hernia' 'abscess of nose'
 'abscess of the pharynx' 'achalasia' 'acne' 'actinic keratosis'
 'acute bronchiolitis' 'acute bronchitis' 'acute bronchospasm']
